In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set_style("whitegrid")
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

In [ ]:
def show_result(mode, pct):
    df = pd.read_csv(f'../input/sls-data-set-results/result{mode}.csv')
    df = df.iloc[:int(df.shape[0]*pct)]
    
    # Plotly
    data = {'Sample': [], 'MSE': []}
    seq_len = int(df.shape[0]/150)
    step = df.iloc[1].Sample - df.iloc[0].Sample
    for indx in range(0, df.shape[0]-seq_len, seq_len):
        data['MSE'].append(df.MSE.iloc[indx:indx+seq_len].mean())
        data['Sample'].append(df.Sample.iloc[indx+seq_len])
    data_plotly = [
        go.Bar(
            x = data['Sample'],
            y = data['MSE'],
            base = 0,
            marker = dict(
            color = data['MSE'],
            colorscale='Reds',
            )
        )
    ]
    layout = go.Layout(
        title=f'MODE{mode}',
        xaxis=dict(title='sample number'),
        yaxis=dict(title='mean squared error'),
    )
    fig = go.Figure(data=data_plotly, layout=layout)
    iplot(fig, filename='color-bar')
    
    # Scatter
    plt.figure(figsize=(20,10))
    data = df.copy()
    data.MSE = data.MSE.rolling(10).mean()
    sns.scatterplot(
        x='Sample',
        y='MSE',
        hue='MSE',
        size='MSE',
        palette=sns.cubehelix_palette(rot=-.2, as_cmap=True),
        sizes=(10, 100),
        data=data,
    )
    
    # Moving Average
    plt.figure(figsize=(20,10))
    data = df.copy()
    data.MSE = data.MSE.rolling(1000).mean().rolling(1000).mean().shift(-1000)
    sns.lineplot(
        x='Sample',
        y='MSE',
        data=data,
    )
    data = df.copy()
    data.MSE = data.MSE.rolling(10000).mean().rolling(10000).mean().shift(-10000)
    sns.lineplot(
        x='Sample',
        y='MSE',
        data=data,
    )

In [ ]:
mode = 5
for i in range(3):
    show_result(mode=mode, pct=1.0-0.2*i)